<a href="https://colab.research.google.com/github/mitali-g/MachineLearning/blob/master/chest_xray_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [4]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import vgg16

In [5]:
!unzip -q dataset-final.zip

In [13]:
trdata = ImageDataGenerator(rescale=1./255)
train_data = trdata.flow_from_directory(directory='dataset-final/train', 
                    target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=True)

vldata = ImageDataGenerator(rescale=1./255)
valid_data = vldata.flow_from_directory(directory='dataset-final/valid', 
                     target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=True)

tsdata = ImageDataGenerator(rescale=1./255)
test_data = tsdata.flow_from_directory(directory='dataset-final/test',
                                       target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=True)

Found 3961 images belonging to 2 classes.
Found 1321 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [9]:
vgg_base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = vgg_base_model.output
x = Flatten()(x)
x = Dense(units=4096, activation='relu')(x)
predictions = Dense(units=2, activation = 'softmax')(x)
model = Model(inputs = vgg_base_model.input, outputs = predictions)

58892288/58889256 [==============================] - 2s 0us/step


In [10]:
for layer in model.layers[:19]:
    layer.trainable = False
for layer in model.layers[19:]:
    layer.trainable = True

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [12]:
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])
step_size_train=train_data.n//train_data.batch_size

model.fit_generator(generator=train_data, steps_per_epoch=step_size_train, 
                    epochs=10, validation_data=valid_data, validation_steps=1)

Epoch 1/10
123/123 [==============================] - 20s 163ms/step - loss: 0.3106 - accuracy: 0.8730 - val_loss: 0.1145 - val_accuracy: 1.0000
Epoch 2/10
123/123 [==============================] - 14s 110ms/step - loss: 0.1668 - accuracy: 0.9476 - val_loss: 0.2330 - val_accuracy: 0.9062
Epoch 3/10
123/123 [==============================] - 14s 111ms/step - loss: 0.1376 - accuracy: 0.9555 - val_loss: 0.0941 - val_accuracy: 0.9688
Epoch 4/10
123/123 [==============================] - 14s 110ms/step - loss: 0.1211 - accuracy: 0.9623 - val_loss: 0.0910 - val_accuracy: 0.9688
Epoch 5/10
123/123 [==============================] - 14s 110ms/step - loss: 0.1072 - accuracy: 0.9672 - val_loss: 0.0342 - val_accuracy: 1.0000
Epoch 6/10
123/123 [==============================] - 13s 109ms/step - loss: 0.1002 - accuracy: 0.9692 - val_loss: 0.0389 - val_accuracy: 1.0000
Epoch 7/10
123/123 [==============================] - 13s 109ms/step - loss: 0.0952 - accuracy: 0.9702 - val_loss: 0.0682 - val_ac

In [17]:
model.evaluate_generator(generator=test_data)

[1.3297070264816284, 0.7003205418586731]